In [1]:
import os
import pickle
import cv2
import torch
from torchvision import transforms
from PIL import Image
import numpy as np
from facenet_pytorch import InceptionResnetV1,MTCNN

In [2]:
model_path = ""
model_state_dict = torch.load(model_path)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = InceptionResnetV1(pretrained='vggface2').to(device)

In [5]:
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [6]:
mtcnn = MTCNN(keep_all=True, min_face_size=20, thresholds=[0.6, 0.7, 0.7])
model.eval()


folder = 'input_face/'
database = {}

# Load existing database if available
db_file = 'face_database.pkl'
if os.path.exists(db_file):
    with open(db_file, 'rb') as f:
        database = pickle.load(f)

for filename in os.listdir(folder):
    path = os.path.join(folder, filename)
    gbr1 = cv2.imread(path)
    
    # Detect faces using MTCNN
    boxes, probs = mtcnn.detect(gbr1)

    if boxes is not None and len(boxes) > 0:
        aligned_faces = mtcnn(gbr1)
        if aligned_faces is not None:
            aligned_faces = aligned_faces.to(device)

            # Generate embeddings using FaceNet
            with torch.no_grad():
                embedding_2 = model(aligned_faces[0].unsqueeze(0)).squeeze(0).cpu().numpy()            
            name = os.path.splitext(filename)[0]
            if name in database:
                database[name].append(embedding_2)
            else:
                database[name] = [embedding_2]
    else:
        print(f"No face detected in {filename}")

# Save the database to a .pkl file
with open(db_file, 'wb') as f:
    pickle.dump(database, f)

print("Database updated and saved to face_database.pkl")

Database updated and saved to face_database.pkl


In [8]:
database

{'ais (2)': [array([-0.02997749,  0.06567424, -0.01415026,  0.00478048, -0.01669426,
         -0.03074986, -0.02105584,  0.01097088,  0.04836982, -0.02619672,
          0.00585723,  0.01070012, -0.018769  ,  0.04918785,  0.02489967,
         -0.05849759, -0.00755017,  0.04443614,  0.03813805, -0.00855652,
         -0.00577787,  0.0226436 ,  0.06048311, -0.0501231 ,  0.02974202,
         -0.05585569,  0.04037178, -0.02362192,  0.05521332, -0.02378276,
          0.09137423,  0.04650403,  0.04486774,  0.07995778,  0.03508594,
         -0.05085004,  0.01435733,  0.00896857, -0.00033304,  0.03098879,
          0.04500034, -0.09060793, -0.01862941,  0.06118779, -0.03659609,
         -0.05664533, -0.04289002,  0.01418536, -0.09289984, -0.02813185,
         -0.00234176, -0.03524841, -0.06439649, -0.09821269, -0.00788092,
          0.01590037, -0.0843263 , -0.00783166,  0.037542  ,  0.01219596,
         -0.08213542,  0.08426826, -0.01123094,  0.04181303,  0.00750527,
         -0.01623653, -0.03